In [ ]:
import numpy as np
import pandas as pd

from gnss_lib_py.parsers.navdata import NavData

In [ ]:
# Get data path of example file
!wget https://raw.githubusercontent.com/Stanford-NavLab/gnss_lib_py/main/notebooks/tutorials/myreceiver.csv --quiet -O "myreceiver.csv"
data_path = "myreceiver.csv"

# Initializing

Create an empty NavData class instantiation

In [ ]:
empty_nav_data = NavData()
print(empty_nav_data)

Create a NavData class from a csv file

In [ ]:
nav_data_csv = NavData(csv_path=data_path)
print(nav_data_csv)

Create an empty NavData class from a pandas DataFrame

In [ ]:
pd_df = pd.read_csv(data_path)
nav_data_pd = NavData(pandas_df=pd_df)
print(nav_data_pd)

Create an empty NavData class instantiation

In [ ]:
np_array = np.eye(4)
nav_data_np = NavData(numpy_array=np_array)
print(nav_data_np)

# Accessing values

In this example, the csv file that we have loaded has the following contents:

| myTimestamp | mySatId | myPseudorange |
| ----------- | ------- | ------------- |
| 10          | G10     | 270000001     |
| 10          | G14     | 270000007     |
| 10          | G07     | 270000004     |
| 10          | G03     | 270000005     |
| 11          | G10     | 270000002     |
| 11          | G14     | 270000008     |
| 11          | G07     | 270000003     |
| 11          | G03     | 270000004     |

`NavData` stores the different types of measurements, like `myTimestamp`
as rows.
So, each row contains the variable at different times/measurements.
In this case, the `myPseudorange` row contains values for different
timestamps and satellite IDs

Values in rows can be accessed by using the name of the row or the
number of the row

In [ ]:
nav_data_csv['myTimestamp']


In [ ]:
nav_data_csv[0]

By specifying a column number (or slice of columns) along with the row label/number, a sub part
of the matrix can be accessed as well

In [ ]:
nav_data_csv['myPseudorange', :3]

To access multiple rows using labels, pass the labels as a list of strings or slice of numbers

In [ ]:
nav_data_csv[['myTimestamp', 'myPseudorange']]

However, to ensure compatability with accessing operations, you  cannot access string and numeric values jointly.
This is to prevent unexpected behaviour that might result on performing mathematical operations with strings and numbers.

If you attempt to access a combination of strings and numbers, the method will return an `AssertionError`

In [ ]:
try:
    nav_data_csv[0:2]
except AssertionError as err:
    print(err)

# Setting values

You can update values for existing rows by indexing those rows and assigning an array

In [ ]:
nav_data_csv['myPseudorange'] = 10*np.arange(8)
nav_data_csv['myPseudorange']

You can also set individual values by accessing the specific rows and columns that you want to reassign

In [ ]:
nav_data_csv['myPseudorange', 2] = 111
nav_data_csv['myPseudorange']


# Setting new row

To set numeric values for a new row, access the row with the label name and assign the value to that row.

In [ ]:
nav_data_csv['new_row'] = np.arange(8)
nav_data_csv['new_row']

New string rows can also be created similarly.

Note that while creating rows with string values, you must set the `dtype` of the row to `object`

In [ ]:
nav_data_csv['new_string_row'] = np.array(['string1', 'string1', 'string1', 'string1',
                                            'string2', 'string2', 'string2', 'string2'], dtype=object)
nav_data_csv['new_string_row']

# Adding new columns

To add new columns, use the `NavData.concat()` method which concatenates two `NavData` instances.

In [ ]:
print(nav_data_np)

In [ ]:
nav_data_np[:]

In [ ]:
nav_data_np.concat(NavData(numpy_array=np_array),inplace=True)
nav_data_np[:]

# Find rows, shape and length

You can see which rows are currently in `NavData` using `NavData.rows`.
The number of columns can be viewed using `len(NavData)` and the shape of the array can be viewed using `NavData.shape`

In [ ]:
nav_data_csv.rows

In [ ]:
nav_data_csv.shape

In [ ]:
len(nav_data_csv)

# Removing rows or columns

To remove rows, use the `NavData.remove()` method, specifying the rows and columns you want to remove.


In [ ]:
nav_data_csv = nav_data_csv.remove(rows=['new_row', 'new_string_row'])
print(nav_data_csv.rows)
print(nav_data_csv.shape)

In [ ]:
nav_data_csv = nav_data_csv.remove(cols=[0, 1])
print(nav_data_csv.shape)

# Copy

If you don't specify any rows or columns, the entire `NavData` will be copied

In [ ]:
nav_data_np_copy = nav_data_np.copy()
print('Original NavData array')
print(nav_data_np[:, :])
print('Copied NavData array')
print(nav_data_np_copy[:, :])

Specifying rows, columns or both copies specified rows and columns.

Note that currently, rows and columns must be specified as lists.

In [ ]:
nav_data_np_copy = nav_data_np.copy(rows=[0,1], cols=list(range(4)))
print('Original NavData array')
print(nav_data_np[:, :])
print('Copied NavData array')
print(nav_data_np_copy[:, :])

# Convert to `pd.DataFrame` and save to csv

You can use the `NavData.save_csv()` method to save the data as a csv file and `NavData.pandas_df()` to get the equivalent `pd.DataFrame`

In [ ]:
nav_data_pd.pandas_df()

# Looping

You can use the `NavData.loop_time()` method to loop over groups of data that belong to same time stamp.
In this case, we will show the data for each timestep using `pandas_df()`, which allows us to show strings and numbers together

In [ ]:
for timestamp, delta_t, small_nav_data in nav_data_pd.loop_time('myTimestamp'):
    print('Current timestamp: ', timestamp)
    print('Difference between current and future time step', delta_t)
    print('Current group of data')
    print(small_nav_data.pandas_df())

You can loop over all columns using in-built looping functions. In this example, we terminate the loop after 3 runs for display clarity

In [ ]:
for col_idx, nav_data_col in enumerate(nav_data_pd):
    print('Current column number', col_idx)
    print('Current column')
    print(nav_data_col.pandas_df())
    if col_idx >= 3:
        break